In [1]:
%matplotlib qt5
from python.analysis import Master, vector, EventDisplay
import apps.prod4a_merge_study as merge_study
import awkward as ak
import numpy as np
import pandas as pd
from tabulate import tabulate 
import matplotlib.pyplot as plt
import matplotlib
from rich import print


def GetSharedHits(pfo1 : ak.Record, pfo2 : ak.Record):
    count = 0
    shared_points = []
    filtered_pfo1 = pfo1[pfo1.x != -999]
    filtered_pfo2 = pfo2[pfo2.x != -999]
    for hitpoint1 in filtered_pfo1:
        for hitpoint2 in filtered_pfo2:
            if(hitpoint1.x == hitpoint2.x and hitpoint1.y == hitpoint2.y and hitpoint1.z == hitpoint2.z):
                count += 1
                shared_points.append(hitpoint1)
                break
    shared_points = ak.Array(shared_points)
    return count, shared_points


def TrueParticleIndices(events, event, particle_number):
    index = ak.local_index(events.trueParticlesBT.number[event])
    return index[events.trueParticlesBT.number[event] == particle_number]


def DrawTrueParticle(points, indices, colour = "black"):
    display.PFO(points[indices[0]], "x", colour, pointSize = 75, alpha = 0.5) # only plot one copy of the MCParticle


def PrintPFOStats(events : Master.Data, event : int, pfo : int):
    mask = events.recoParticles.number[event] == pfo
    dict = {
        "number": [pfo],
        "backtracked particle": events.trueParticlesBT.number[event][mask],
        "energy": events.recoParticles.energy[event][mask],
        "collection hits": events.recoParticles.nHits[event][mask],
        "shared collection hits": events.trueParticlesBT.sharedHits_collection[event][mask],
        "purity": events.trueParticlesBT.purity[event][mask],
        "completeness": events.trueParticlesBT.completeness[event][mask]
    }
    print(tabulate(dict, headers = dict.keys(), tablefmt="latex"))


def PrintTruePFOStats(events : Master.Data, event : int, particle):
    mask = events.trueParticlesBT.number[event] == particle
    dict = {
        "number": [particle],
        "energy": events.trueParticlesBT.energy[event][mask],
        "true collection hits": events.trueParticlesBT.nHits_collection[event][mask],
    }
    print(tabulate(dict, headers = dict.keys(), tablefmt="latex"))


def PlotHits(events : Master.Data, event : int, particle_number : int, reco_points : ak.Record, true_points : ak.Record, reco_colours : np.array, true_colour : str):    
    i = TrueParticleIndices(events, event, particle_number)

    print(f"Fragments:{ak.count(i)}")
    print(f"true PFO hits: {ak.count(true_points[event][i[0]]) / 3}")

    DrawTrueParticle(true_points[event], i, colour = true_colour)
    PrintTruePFOStats(events, event, particle_number)

    for j in range(len(i)):
        p = reco_points[event][i[j]]
        n = ak.count(p) / 3 # divide by 3 because it counts all components of the vector
        print(f"Fragment hits: {n}")
        if (n == 0): continue
        c = matplotlib.colors.rgb2hex(reco_colours[j % len(reco_colours)])
        display.PFO(p, "+", c, pointSize = 100, alpha = 1)

        textPosition = p[p.x != -999][0]
        display.Text(textPosition, events.recoParticles.number[event][i[j]], colour = c)

        PrintPFOStats(events, event, events.recoParticles.number[event][i[j]])
    return


def PlotPFOsInSlice(events : Master.Data, event : int, particle_number : int, reco_points : ak.Record, true_points : ak.Record, reco_colours : np.array, true_colour : str):
    i = TrueParticleIndices(events, event, particle_number)
    DrawTrueParticle(true_points[event], i, true_colour)
    PrintTruePFOStats(events, event, particle_number)

    slices = np.unique(events.recoParticles.sliceID[event][events.trueParticlesBT.number[event] == particle_number])
    print(f"slices: {slices}")

    for s in slices:
        pfos = ak.local_index(events.recoParticles.number[event])[events.recoParticles.sliceID[event] == s]
        for i in pfos:
            p = reco_points[event][i]
            if(ak.count(p) == 0):
                continue
            c = matplotlib.colors.rgb2hex(reco_colours[i % len(reco_colours)])
            display.PFO(p, "+", colour = c, pointSize = 100, alpha = 1)

            textPosition = p[p.x != -999][0]
            display.Text(textPosition, events.recoParticles.number[event][i], colour = c)
            PrintPFOStats(events, event, events.recoParticles.number[event][i])
    return

In [2]:
events = Master.Data("work/ROOTFiles/Prod4a_1GeV_BeamSim_00_PFO_study.root")
events.io.ListNTuples("spacePoint")

events.trueParticlesBT.true_points = vector.vector(
    events.io.Get("reco_daughter_PFP_true_byHits_spacePointX"),
    events.io.Get("reco_daughter_PFP_true_byHits_spacePointY"),
    events.io.Get("reco_daughter_PFP_true_byHits_spacePointZ")
)

events.recoParticles.reco_points = vector.vector(
    events.io.Get("reco_daughter_allShower_spacePointX"),
    events.io.Get("reco_daughter_allShower_spacePointY"),
    events.io.Get("reco_daughter_allShower_spacePointZ")
)

print(events.trueParticles.PrimaryPi0Mask[events.trueParticles.PrimaryPi0Mask])

merge_study.EventSelection(events)
# merge_study.PFOSelection(events)
start_showers, to_merge = merge_study.SplitSample(events)

[
    'reco_daughter_allShower_spacePointX',
    'reco_daughter_allShower_spacePointY',
    'reco_daughter_allShower_spacePointZ',
    'reco_daughter_PFP_true_byHits_spacePointX',
    'reco_daughter_PFP_true_byHits_spacePointY',
    'reco_daughter_PFP_true_byHits_spacePointZ'
]

[[], [True], [True], [], [], [], [], ... [True], [], [True], [], [True], [], []]

/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:54: UserWarning: Couldn't apply filters to filters.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:54: UserWarning: Couldn't apply filters to _TrueParticleData__pi0_MC.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:54: UserWarning: Couldn't apply filters to filename.
  warnings.warn(f"Couldn't apply filters to {var}.")


'BeamMCFilter' executed in 0.0285s

number of dalitz decays: 0

...

False

True

True

True

'SplitSample' executed in 0.2768s

In [3]:
event = 10
nPFO = ak.num(events.recoParticles.number)[event]

pi0_showers = (events.trueParticlesBT.number[np.logical_or(*start_showers)])[event]
print(f"pi0 showers: {pi0_showers}")

uniqueParticles = np.unique(events.trueParticlesBT.number[event])
uniqueParticles = uniqueParticles[uniqueParticles != -999]
print("list of PFOs to plot:")
print(ak.to_list(uniqueParticles))

pi0 showers: [633011, 633012]

list of PFOs to plot:

[
    1,
    2,
    20,
    2684,
    2717,
    2753,
    2792,
    2808,
    2812,
    2827,
    2866,
    2870,
    2881,
    2890,
    2903,
    2919,
    2923,
    2938,
    2940,
    2959,
    2965,
    2979,
    2985,
    3021,
    3052,
    3058,
    3067,
    3087,
    3089,
    3152,
    3161,
    3162,
    3186,
    3232,
    3256,
    3284,
    3293,
    3302,
    3323,
    3330,
    3358,
    3378,
    89532,
    189566,
    268137,
    292548,
    357537,
    378132,
    378174,
    393011,
    460245,
    504862,
    581296,
    582781,
    587124,
    633001,
    633011,
    633012
]

In [4]:
n = 0
reco_display_option = "all"
name = f"{event}_{n}_{reco_display_option}.png"
print(name)

number = pi0_showers[n]
display = EventDisplay.EventDisplay(events.eventNum[event], events.run[event], events.subRun[event], plot2D = True, plot3D = True)
display.xy.grid()
display.xz.grid()

def GenerateColours(name, n):
    return matplotlib.cm.get_cmap(name)(np.linspace(0, 1, n))

c1 = GenerateColours("autumn", 5)
c2 = GenerateColours("cool", 10)
c3 = GenerateColours("hsv", 20)

match reco_display_option:
    case "bt":
        PlotHits(events, event, number, events.recoParticles.reco_points, events.trueParticlesBT.true_points, c2, true_colour = "black")
    case "slice":
        PlotPFOsInSlice(events, event, number, events.recoParticles.reco_points, events.trueParticlesBT.true_points, c2, "black")
    case "all":
        i = TrueParticleIndices(events, event, number)
        DrawTrueParticle(events.trueParticlesBT.true_points[event], i)
        for i in range(nPFO):
            display.PFO(events.recoParticles.reco_points[event][i], "+", c3[i % len(c3)], pointSize = 100)
    case "shower overlay":
        s_0 = TrueParticleIndices(events, event, pi0_showers[0])
        s_1 = TrueParticleIndices(events, event, pi0_showers[1])
        DrawTrueParticle(events.trueParticlesBT.true_points[event], s_0, colour = "black")
        DrawTrueParticle(events.trueParticlesBT.true_points[event], s_1, colour = "cyan")
        counts, hits = GetSharedHits(events.trueParticlesBT.true_points[event][s_0[0]], events.trueParticlesBT.true_points[event][s_1[0]])
        display.PFO(hits, marker = "v", colour = "pink", pointSize = 75)



display.xy.legend(labels = [f"true particle: {number}"])
display.DetectorBounds()
# display.DetectorBounds((-160, -90), (100, 400), (140, 240))
plt.show()

10_0_all.png

In [5]:
print(counts)

NameError: name 'counts' is not defined